In [2]:
import json
from igraph import Graph

# Leer y parsear el archivo JSON
with open('../input/tests.json', 'r') as f:
    data = json.load(f)

# Extraer nodos y aristas del JSON
nodes = data['nodes']
edges = data['edges']

# Crear un grafo bipartito vacío
g = Graph()

# Añadir nodos al grafo, con el atributo bipartite
for node in nodes:
    g.add_vertex(name=node['id'], label=node['label'], type=node['type'])

# Añadir aristas al grafo y sus atributos
for edge in edges:
    g.add_edge(edge['source'], edge['target'], weight=edge['weight'], relation=edge['relation'])

# Marcar el grafo como bipartito
# La propiedad 'type' de los nodos se utilizará para identificar las dos partes
g.vs["type"] = [node["type"] for node in nodes]

# Mostrar información del grafo
print(g.summary())

# Verificar si el grafo es bipartito
is_bipartite = g.is_bipartite()
print("El grafo es bipartito:", is_bipartite)

# Mostrar los atributos de las aristas
for edge in g.es:
    print(f"Arista entre {edge.source} y {edge.target} con peso {edge['weight']} y relación {edge['relation']}")


IGRAPH UNWT 4 2 -- 
+ attr: label (v), name (v), type (v), relation (e), weight (e)
El grafo es bipartito: True
Arista entre 0 y 1 con peso 1.5 y relación friend
Arista entre 2 y 3 con peso 2.5 y relación colleague


C:\Users\bcriv\AppData\Local\Temp\ipykernel_12256\497454828.py:17: DeprecationWarning: You are using integers as vertex names. This is discouraged because most igraph functions interpret integers as vertex _IDs_ and strings as vertex names. For sake of consistency, convert your vertex names to strings before assigning them. Future versions from igraph 0.11.0 will disallow integers as vertex names.
  g.add_vertex(name=node['id'], label=node['label'], type=node['type'])


In [4]:
g.vs['name']

[0, 1, 2, 3]